In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.firefox.service import Service
import random
from dotenv import load_dotenv
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from selenium.common.exceptions import NoSuchElementException
import sqlite3
import os
import pandas as pd
from selenium.common.exceptions import TimeoutException

In [ ]:
def driver_setting():
    driver_path = "./geckodriver"
    options = webdriver.FirefoxOptions()
    # options.add_argument('--headless')
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(
        "user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")

    service = Service(executable_path=driver_path)
    driver = webdriver.Firefox(service=service, options=options)

    return driver


def sleep_time(min_sec, max_sec):
    time.sleep(random.uniform(min_sec, max_sec))
    driver.implicitly_wait(random.uniform(min_sec, max_sec))


def find_n_click_xpath(xpath, min=1.4, max=2):
    element = driver.find_element(By.XPATH, xpath)
    element.click()
    sleep_time(min, max)


def find_n_get_text_xpath(xpath):
    try:
        texted = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, xpath))).text
        if texted == "":
            texted = None
        return texted
    except TimeoutException:
        return None


def find_n_get_text_xpath_for_scores(xpath):
    try:
        texted = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        ).text
        if texted == "":
            return "not_found"
        return texted
    except TimeoutException:
        return "not_found"


def get_href(xpath):
    element = driver.find_element(By.XPATH, xpath)
    href = element.get_attribute("href")
    return href


def everytime_login(id, pw):
    driver.get("https://account.everytime.kr/login")
    sleep_time(1, 2)

    driver.find_element(By.NAME, 'id').send_keys(id)
    driver.find_element(By.NAME, 'password').send_keys(pw)
    sleep_time(0.3, 1)

    driver.find_element(
        By.CSS_SELECTOR, '.container > form:nth-child(2) > input:nth-child(2)').click()
    sleep_time(0.5, 1)


def find_n_get_review_page(lecNum):
    driver.get('https://everytime.kr/timetable/2024/2')
    sleep_time(1, 2)

    find_n_click_xpath("/html/body/div[1]/ul/li[1]")  # 수업목록에서 검색 버튼
    find_n_click_xpath("/html/body/div[2]/div[1]/a[4]", 0.3, 0.8)  # 검색어 버튼
    find_n_click_xpath(
        "/html/body/div[1]/form[5]/div/label[3]", 0.3, 0.8)  # 과목 코드 버튼

    input_element = driver.find_element(
        By.XPATH, '/html/body/div[1]/form[5]/div/input')  # input에 클릭
    input_element.send_keys(lecNum)  # 값 넣기
    sleep_time(0.5, 1)

    find_n_click_xpath("/html/body/div[1]/form[5]/input")
    sleep_time(1, 2)

    try:
        a_path = driver.find_element_by_xpath(
            "/html/body/div[2]/div[2]/table/tbody/tr/td[7]/a")
        href = a_path.get_attribute('href')
    except NoSuchElementException:
        href = None

    return href


def insert_review_and_update_date(lecture_id, reviewer, review_star, review):
    conn = sqlite3.connect(database)
    cursor = conn.cursor()

    insert_query = """
    INSERT INTO LectureReview (lectureID, reviewer, reviewStar, review)
    VALUES (?, ?, ?, ?);
    """
    update_query = """
    UPDATE LectureEverytimeData
    SET crawling_date_int = 241119
    WHERE LectureID = ?;
    """

    cursor.execute(insert_query, (lecture_id, reviewer, review_star, review))
    cursor.execute(update_query, (lecture_id,))
    conn.commit()

    cursor.close()
    conn.close()


########################################################################


database = "./kwu-lecture-recommendation-v8.db"
conn = sqlite3.connect(database)
cursor = conn.cursor()

# 1학기
query = """
SELECT LectureID, everytimeUrl
FROM LectureEverytimeData
WHERE everytimeUrl IS NOT NULL
  AND crawling_date_int < 241119;
"""

cursor.execute(query)

lectureIDS = cursor.fetchall()

print(lectureIDS)
cursor.close()
conn.close()


driver = driver_setting()

load_dotenv()

everytime_login(os.getenv("etid"), os.getenv("etpw"))

for rows in lectureIDS:
    lectureID, everytimeUrl = rows
    print("|------|----------------------------------------------------------------------------")
    try:
        start_time = time.time()
        print(
            f"|      | crawling (lectureID: {lectureID}, lecNum: {everytimeUrl}) ")
        url = everytimeUrl + "?tab=article"
        driver.get(url)

        print(f"| good | moved to {url}")

        sleep_time(1, 2)
        articles = driver.find_element(By.CLASS_NAME, 'articles')

        last_height = driver.execute_script(
            "return arguments[0].scrollHeight", articles)

        while True:
            driver.execute_script(
                "arguments[0].scrollTop = arguments[0].scrollHeight", articles)
            sleep_time(0.5, 1)

            new_height = driver.execute_script(
                "return arguments[0].scrollHeight", articles)
            if new_height == last_height:
                break
            last_height = new_height

        semester_elements = articles.find_elements(By.CLASS_NAME, 'semester')
        semesters_text = [elem.text for elem in semester_elements]

        on_elements = articles.find_elements(By.CLASS_NAME, 'on')
        on_widths = [int(elem.value_of_css_property('width').replace(
            'px', '').strip()) for elem in on_elements]

        articles_texts = articles.find_elements(By.CLASS_NAME, "text")
        review_texts = [elem.text for elem in articles_texts]

        for i in range(len(review_texts)):
            if i >= len(semesters_text) or i >= len(on_widths):
                print(
                    f"!! Index out of range for lectureID: {lectureID}. Exiting loop.")
                break

            reviewer = semesters_text[i]
            review_star = on_widths[i]
            review = review_texts[i]

            insert_review_and_update_date(
                lectureID, reviewer, review_star, review)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"| good | crawling {len(semesters_text)}, {
              elapsed_time:.2f} sec")

    except Exception as e:
        end_time = time.time()
        print(f"\n!! error !! at lectureID: {lectureID[0]}\n{e}\n\n")

[(6, 'https://everytime.kr/lecture/view/2736225'), (7, 'https://everytime.kr/lecture/view/1936761'), (8, 'https://everytime.kr/lecture/view/2249026'), (9, 'https://everytime.kr/lecture/view/1611476'), (10, 'https://everytime.kr/lecture/view/2470736'), (11, 'https://everytime.kr/lecture/view/2009295'), (12, 'https://everytime.kr/lecture/view/2470735'), (13, 'https://everytime.kr/lecture/view/2086551'), (14, 'https://everytime.kr/lecture/view/2314909'), (15, 'https://everytime.kr/lecture/view/2571077'), (16, 'https://everytime.kr/lecture/view/2314909'), (17, 'https://everytime.kr/lecture/view/2736227'), (18, 'https://everytime.kr/lecture/view/2376443'), (19, 'https://everytime.kr/lecture/view/2571078'), (20, 'https://everytime.kr/lecture/view/2736226'), (21, 'https://everytime.kr/lecture/view/1611472'), (22, 'https://everytime.kr/lecture/view/2658166'), (23, 'https://everytime.kr/lecture/view/2470737'), (24, 'https://everytime.kr/lecture/view/2009297'), (25, 'https://everytime.kr/lecture

TypeError: 'int' object is not subscriptable

`url +"?tab=article"`
